# Encoder_Decoder Simplified Data

Q: 1. Data filter? Recognized? 

In [91]:
'''Author: Ding Shuya 
Description: This doc is used for showing encoder-decoder
'calender', 'snowman', 'penguin', 'blackberry', 'teddy-bear'
'JP', 'CN', 'DE'
'''

"Author: Ding Shuya \nDescription: This doc is used for showing encoder-decoder\n'calender', 'snowman', 'penguin', 'blackberry', 'teddy-bear'\n'JP', 'CN', 'DE'\n"

In [1]:
import torch
import torch.nn as nn
import os
root_path = '/export/home/di0002ya/quickdraw/cnn_base'
data_path = '/export/home/di0002ya/quickdraw/data/sy_data/quick_draw/'
out_data_path = '/export/home/di0002ya/quickdraw/data/sy_data/quick_draw_output/'
os.chdir(root_path)
from feature_engineering_func_ntu import *

## 1. Data Preprocessing Simple 

In [14]:
import pickle

with open(os.path.join("/export/home/di0002ya/quickdraw/toy_dataset/train_X"), "rb") as f:
    train_X = pickle.load(f)
with open(os.path.join("/export/home/di0002ya/quickdraw/toy_dataset/train_Y"), "rb") as f:
    train_Y = pickle.load(f)
with open(os.path.join("/export/home/di0002ya/quickdraw/toy_dataset/test_X"), "rb") as f:
    test_X = pickle.load(f)
with open(os.path.join("/export/home/di0002ya/quickdraw/toy_dataset/test_Y"), "rb") as f:
    test_Y = pickle.load(f)

In [57]:
# Create df
train_X_df = pd.DataFrame(train_X)
train_Y_df = pd.DataFrame(train_Y)
test_X_df = pd.DataFrame(test_X)
test_Y_df = pd.DataFrame(test_Y)

In [63]:
# Find Max Stroke Per Drawings 
df = train_X_df 
Nmax = max(train_X_df.shape[1], test_X_df.shape[1])
print(Nmax)

60


In [59]:

def _array_normalizer(array1,Xmin,Xmax,array_min):
    '''
    function:
        - normalize X,Y array by range of X
        - used in feature_eng_pt2
    input:
        array1 = array that you want to normalize (1D array or list)
        Xmin = minimum value of your X array (int)
        Xmax = maximum value of your X array (int)
        array_min = minimum value of array1

    output:
        normalized array of array1
    '''
    return (np.array(array1)-np.array([array_min]*len(array1)))/float(Xmax-Xmin)



In [60]:
def rnn_cnn(df_cf, ls): 
    df_cf.index = range(len(df_cf))
    Ymax_list = []
    image_list = []
    # Add Stroke Number Info
    len_info = []
    for element in ls:
        len_info.append(len(element))
    df_cf['stroke_number'] = pd.DataFrame(len_info)
    #ith Drawings
    for i in df_cf.index:
        num = df_cf.loc[i,'stroke_number']
        #jth Stroke
        X = []
        Y = []
        Ymax = []
        ttnum_dp = []
        Tdifftemp = []
        for stroke in range(num):
            stroke_list_X = df_cf.loc[i][stroke][0] 
            stroke_list_Y = df_cf.loc[i][stroke][1] 
            Xmintemp = np.min(stroke_list_X)-10
            Xmaxtemp = np.max(stroke_list_X)+10
            Ymintemp = np.min(stroke_list_Y)-10
            Xnorm = _array_normalizer(stroke_list_X, Xmintemp,Xmaxtemp,Xmintemp)
            Ynorm = _array_normalizer(stroke_list_Y, Xmintemp,Xmaxtemp,Ymintemp)
            Ymax.append(np.max(Ynorm))
            X.append(Xnorm)
            Y.append(Ynorm)
            ttnum_dp.append(len(Ynorm))
            Tdifftemp.append(1)
            image_pile = np.zeros((num,1176))
            image = np.zeros((42,28))
            xarray = np.around(np.array(X[stroke])*28)
            yarray = np.around(np.array(Y[stroke])*42/float(Ymax[stroke]))
            xarray[xarray>=28.] = 27
            yarray[yarray>=42.] = 41
            for item in range(len(xarray)):
                image[int(np.around(yarray[item])),int(np.around(xarray[item]))] = Tdifftemp[stroke]
            image_pile[stroke] = image.reshape(1,1176)
        Ymax_list.append(Ymax)
        image_list.append(image_pile)
    return Ymax_list, image_list

In [61]:
Train_Ymax_list, Train_image_list = rnn_cnn(train_X_df, train_X)

In [62]:
Test_Ymax_list, Test_image_list = rnn_cnn(test_X_df, test_X)

In [67]:
def pad_list(Nmax, image_list):
    pad_width = Nmax 
    image_full_list = []
    for i in range(len(image_list)):
        pad_ar = np.zeros((pad_width - image_list[i].shape[0],1176))
        image_full_list.append(np.vstack((image_list[i],pad_ar)))
    image_full_ar = np.array(image_full_list)
    return image_full_ar

In [68]:
Train_image_full_ar = pad_list(Nmax,Train_image_list)
Test_image_full_ar = pad_list(Nmax,Test_image_list)

In [69]:
print(Train_image_full_ar.shape)

(26758, 60, 1176)


In [86]:
print(Test_image_full_ar.shape)

(6690, 60, 1176)


In [87]:
#Reshape input to (seq_len, batch, input_size)
train_dset = np.reshape(Train_image_full_ar, (Train_image_full_ar.shape[1],
                                              Train_image_full_ar.shape[0],Train_image_full_ar.shape[2]))
train_dset = torch.from_numpy(train_dset).float()
test_dset = np.reshape(Test_image_full_ar, (Test_image_full_ar.shape[1],Test_image_full_ar.shape[0],
                                       Test_image_full_ar.shape[2]))
test_dset = torch.from_numpy(test_dset).float()

In [88]:
print(train_dset.shape)

torch.Size([60, 26758, 1176])


In [89]:
#Reshape Labels 
train_label = torch.from_numpy(np.array(train_Y))
print(train_label.shape)
test_label = torch.from_numpy(np.array(test_Y))
print(test_label.shape)

torch.Size([26758])
torch.Size([6690])


In [90]:
#Split 0.1 Training Data as Evaluation Datasets 
val_label = train_label[0:2000]
train_rem_label = train_label[2000:]
val_dset = train_dset[:,0:2000,:]
train_rem_dset = train_dset[:,2000:,:]

In [ ]:
# Setting Tune Parameters 
args={}
args['epoch'] = 2
args['hidden_size'] = 4000
args['batch_size'] = 512
args['lr'] = 5
args['output_size'] = 5
args['mid_size'] = 4096

In [125]:
class encoder_decoder_1(nn.Module):
    def __init__(self, hidden_size, init_size, mid_size,output_size):
        super(encoder_decoder_1, self).__init__()
        self.encLSTM1 = nn.LSTM(init_size, hidden_size)
        self.encLSTM2 = nn.LSTM(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, mid_size)
        self.decConv1 = nn.Conv2d(1, 1, kernel_size = 4, padding=0, stride=2)
        self.decConv2 = nn.Conv2d(1, 1, kernel_size = 3, padding=1, stride=2)
        self.output = nn.Linear(256, output_size)
    def forward(self, h_init, c_init, word_seq):
        h_seq, (h_final,c_final) = self.encLSTM1(word_seq, (h_init,c_init))
        score_final = self.linear(h_final)
        score_final = score_final.view(score_final.shape[1],1,64,64)
        x = self.decConv1(score_final)
        x = self.decConv2(x)
        x = x.view(x.shape[0],x.shape[1]*x.shape[2]*x.shape[3])
        print(x)
        y = self.output(x)
        print(y)
        return y

In [126]:
net = encoder_decoder_1(args['hidden_size'],1176,args['mid_size'],args['output_size'])
print(net)

encoder_decoder_1(
  (encLSTM1): LSTM(1176, 4000)
  (encLSTM2): LSTM(4000, 4000)
  (linear): Linear(in_features=4000, out_features=4096, bias=True)
  (decConv1): Conv2d(1, 1, kernel_size=(4, 4), stride=(2, 2))
  (decConv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (output): Linear(in_features=256, out_features=5, bias=True)
)


In [127]:
device = torch.device('cuda')
print(device)

cuda


In [96]:
mean = train_dset.mean()
std = train_dset.std()

In [98]:
# GPU Versions..
# net = net.to(device)
# mean = mean.to(device)
# std = std.to(device)
# print(torch.__version__)
# net.cuda()

In [99]:
# Define Loss
criterion = nn.CrossEntropyLoss()
# Define Learning Rate 
my_lr = args['lr']

In [116]:
def eval_on_test_set(args, net, val_label, val_dset, device, mean, std):
    running_error = 0
    num_batches = 0 
    for i in range(0,test_label.shape[0]):
        minibatch_data = val_dset[:,count:count+bs,:]
        minibatch_label = val_label[count:count+bs]
        # Send to gpu 
#         minibatch_data = minibatch_data.to(device)
#         minibatch_label = minibatch_label.to(device)
        # Normalized minibatch data
        inputs = (minibatch_data - mean) /std
        # Compute output 
        h = torch.zeros(1, args['batch_size'], args['hidden_size'])
        c = torch.zeros(1, args['batch_size'], args['hidden_size'])
        h = h.detach()
        c = c.detach()
#         # Send to gpu
#         h = h.to(device)
#         c = c.to(device)
        inputs.requires_grad_()
        y = net(h, c, inputs)
        error = get_error(y.detach(), minibatch_label)
        running_error += error.item()
        num_batches +=1
    total_error = running_error/num_batches
    print('error rate on test set =', total_error*100,'percent')
    return total_error 

def train_enc_dec(args, net, my_lr, train_label, train_dset, device, mean, std, output,name, val_label, val_dset):
    create_dir(output)
    best_error = 1000000
    logger = Logger(os.path.join(output,'log.txt'))
    for epoch in range(args['epoch']):
        print('strat new epoch')
        bs = args['batch_size']
        hidden_size = args['hidden_size']
        if epoch >= 2:
            my_lr = my_lr/3
        optimizer = torch.optim.SGD(net.parameters(), my_lr)
        running_loss = 0
        running_error = 0
        num_batches = 0 
        h = torch.zeros(1, bs, hidden_size)
        c = torch.zeros(1, bs, hidden_size)
        h = h.detach()
        c = c.detach()
#         # Send to gpu
#         h = h.to(device)
#         c = c.to(device)
        shuffled_indices = torch.randperm(train_label.shape[0])
        for count in range(0,train_label.shape[0],bs):
            print('strat new batch')
            # Create Random MiniBatch
            indices = shuffled_indices[count:count+bs]
            minibatch_data = train_dset[:,indices,:]
            minibatch_label = train_label[indices]
            # Send to gpu 
#             minibatch_data = minibatch_data.to(device)
#             minibatch_label = minibatch_label.to(device)
            # Normalized minibatch data
            inputs = (minibatch_data - mean) /std
            # Compute output 
            inputs.requires_grad_()
            y = net(h, c, inputs)
            # Compute Avg Loss
            loss = criterion(y, minibatch_label.view(-1).long())
            # backward 
            loss.backward()
            # One step sgd
            optimizer.step()
            # Computing loss
            running_loss += loss.detach().item()
            # Computing error 
            error = get_error(y.detach(), minibatch_label)
            running_error += error.item()

            num_batches +=1
        #Compute stats for the full training set
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches
        elapsed = (time.time()-start)/60
    
    
        logger.write('epoch %d, time: %.2f' % (epoch, elapsed))
        logger.write('\t lr: %.3f' % (my_lr))
        logger.write('\t train loss: %.3f, train_error: %.3f' % (total_loss, total_error*100))
        
        net.train(False)
        eval_error = eval_on_test_set(args, net, val_label, val_dset, device, mean, std)
        net.train(True)
        print('')
        record = time.time()
        if eval_error<best_error:
            print('this epoch good, saved models')
            model_pth = str(record)+'_'+name+'_EncDecmodel.pth'
            torch.save(net,os.path.join(output,model_pth))
            best_error = eval_error

from __future__ import print_function

import errno
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn


EPS = 1e-7


def assert_eq(real, expected):
    assert real == expected, '%s (true) vs %s (expected)' % (real, expected)


def assert_array_eq(real, expected):
    assert (np.abs(real-expected) < EPS).all(), \
        '%s (true) vs %s (expected)' % (real, expected)


def load_folder(folder, suffix):
    imgs = []
    for f in sorted(os.listdir(folder)):
        if f.endswith(suffix):
            imgs.append(os.path.join(folder, f))
    return imgs


def load_imageid(folder):
    images = load_folder(folder, 'jpg')
    img_ids = set()
    for img in images:
        img_id = int(img.split('/')[-1].split('.')[0].split('_')[-1])
        img_ids.add(img_id)
    return img_ids


def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def weights_init(m):
    """custom weights initialization."""
    cname = m.__class__
    if cname == nn.Linear or cname == nn.Conv2d or cname == nn.ConvTranspose2d:
        m.weight.data.normal_(0.0, 0.02)
    elif cname == nn.BatchNorm2d:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    else:
        print('%s is not initialized.' % cname)


def init_net(net, net_file):
    if net_file:
        net.load_state_dict(torch.load(net_file))
    else:
        net.apply(weights_init)


def create_dir(path):
    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise


class Logger(object):
    def __init__(self, output_name):
        dirname = os.path.dirname(output_name)
        if not os.path.exists(dirname):
            os.mkdir(dirname)

        self.log_file = open(output_name, 'w')
        self.infos = {}

    def append(self, key, val):
        vals = self.infos.setdefault(key, [])
        vals.append(val)

    def log(self, extra_msg=''):
        msgs = [extra_msg]
        for key, vals in self.infos.iteritems():
            msgs.append('%s %.6f' % (key, np.mean(vals)))
        msg = '\n'.join(msgs)
        self.log_file.write(msg + '\n')
        self.log_file.flush()
        self.infos = {}
        return msg

    def write(self, msg):
        self.log_file.write(msg + '\n')
        self.log_file.flush()
        print(msg)
        
def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels.view(-1) == labels.view(-1).long())
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs    


In [128]:
# Build AN Function for Hyperparameter Tuning 
create_dir(output)
best_error = 1000000
bs = args['batch_size']
hidden_size = args['hidden_size']
optimizer = torch.optim.SGD(net.parameters(), my_lr)
running_loss = 0
running_error = 0
num_batches = 0 
h = torch.zeros(1, bs, hidden_size)
c = torch.zeros(1, bs, hidden_size)
h = h.detach()
c = c.detach()
#         # Send to gpu
#         h = h.to(device)
#         c = c.to(device)
shuffled_indices = torch.randperm(train_label.shape[0])
count = 0 
print('strat new batch')
# Create Random MiniBatch
indices = shuffled_indices[count:count+bs]
minibatch_data = train_dset[:,indices,:]
minibatch_label = train_label[indices]
# Send to gpu 
#             minibatch_data = minibatch_data.to(device)
#             minibatch_label = minibatch_label.to(device)
# Normalized minibatch data
inputs = (minibatch_data - mean) /std
# Compute output 
inputs.requires_grad_()
y = net(h, c, inputs)
# Compute Avg Loss
loss = criterion(y, minibatch_label)
print(y)
print(minibatch_label)
# backward 
loss.backward()
# One step sgd
optimizer.step()
# Computing loss
running_loss += loss.detach().item()
print(running_loss)
# Computing error 
error = get_error(y.detach(), minibatch_label)
running_error += error.item()
print(running_error)

strat new batch
tensor([[-0.2543, -0.1737, -0.1745,  ..., -0.1410, -0.1448, -0.0811],
        [-0.2543, -0.1737, -0.1745,  ..., -0.1410, -0.1448, -0.0811],
        [-0.2544, -0.1737, -0.1746,  ..., -0.1410, -0.1448, -0.0811],
        ...,
        [-0.2513, -0.1742, -0.1752,  ..., -0.1395, -0.1465, -0.0829],
        [-0.2543, -0.1737, -0.1745,  ..., -0.1410, -0.1448, -0.0811],
        [-0.2543, -0.1737, -0.1745,  ..., -0.1410, -0.1448, -0.0811]],
       grad_fn=<ViewBackward>)
tensor([[ 0.1274,  0.1057,  0.0389,  0.1048, -0.1227],
        [ 0.1274,  0.1057,  0.0389,  0.1048, -0.1227],
        [ 0.1274,  0.1057,  0.0390,  0.1048, -0.1227],
        [ 0.1274,  0.1057,  0.0389,  0.1048, -0.1227],
        [ 0.1274,  0.1057,  0.0389,  0.1048, -0.1228],
        [ 0.1274,  0.1057,  0.0389,  0.1048, -0.1227],
        [ 0.1275,  0.1058,  0.0389,  0.1049, -0.1227],
        [ 0.1278,  0.1061,  0.0398,  0.1057, -0.1235],
        [ 0.1274,  0.1057,  0.0389,  0.1048, -0.1227],
        [ 0.1274,  0.105

In [119]:
# output = '/export/home/di0002ya/quickdraw/data/sy_data/model/'
# train_enc_dec(args=args, net=net, my_lr=my_lr, train_label=train_label,train_dset=train_dset, 
#               device=device, mean=mean, std=std, output=output,name='simple', val_label=val_label, val_dset=val_dset)